In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask
!pip install pyngrok
!pip install flask-cors
!ngrok authtoken 2dQz9vgx2sRTgxs4fSymM2ANuoF_2K6xy4APgChmC4NtStL8H


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install transformers
# !pip install fastt5

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [5]:
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer

# Read and preprocess the JSON file
def read_json_file(json_file):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

# Preprocess text
def preprocess_text(text):
    # Implement your text preprocessing steps here
    # For example, remove stopwords, punctuation, etc.
    return text

# Tokenization and TF-IDF calculation
def extract_keywords(texts):
    vectorizer = TfidfVectorizer(max_features=10000000)  # You can adjust max_features as needed
    tfidf_matrix = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

def get_keywords_for_files(json_data):
    texts = [entry["text"] for entry in json_data["results"]]
    tfidf_matrix, feature_names = extract_keywords(texts)

    keywords_per_file = {}
    for i, entry in enumerate(json_data["results"]):
        filename = entry["file"]
        text = entry["text"]
        preprocessed_text = preprocess_text(text)
        tfidf_scores = tfidf_matrix[i].toarray().flatten()  # Use index i as row index
        keywords_indices = tfidf_scores.argsort()[-100000000000:]  # You can adjust the number of keywords to extract
        keywords = [feature_names[idx] for idx in keywords_indices]
        keywords_per_file[filename] = { "keywords" : keywords , "text" : text }

    return keywords_per_file


# Matching function between question and filenames based on keywords
def find_related_files(question, keywords_per_file):
    related_files = []
    question_keywords = preprocess_text(question).split()  # Preprocess the question and extract keywords

    for filename, keywords in keywords_per_file.items():
        intersection = set(keywords["keywords"]) & set(question_keywords)
        if len(intersection) > 2:
            related_files.append({"related_files":filename, "text":keywords["text"]})

    return related_files

# Main function to orchestrate the process
def main(json_file, question):
    # Read JSON file
    json_data = read_json_file(json_file)

    # Extract keywords for each file
    keywords_per_file = get_keywords_for_files(json_data)

    # Match question with related files based on keywords
    related_files = find_related_files(question, keywords_per_file)

    return related_files

# # Example usage
# if __name__ == "__main__":
#     json_file = "your_json_file.json"
#     question = "your_question_here"
#     related_files = main(json_file, question)
#     print(related_files)




In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def t5_question_answering(context, question):
    # Load pre-trained T5 model and tokenizer
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Prepare input
    input_text = "question: {} context: {}".format(question, context)
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate answer
    output = model.generate(input_ids=input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    return answer

In [7]:
json_file = "/content/drive/My Drive/content-chatbot-txt-files/repots.json"
json_data = read_json_file(json_file)
keywords_per_file = get_keywords_for_files(json_data)


# question ="protokoll kommunfullmäktige 2024-02-29 signatur näe ?"
# related_files  = find_related_files(question, keywords_per_file)
# print("Related files:", related_files)

# content =  related_files[0]["text"]
# answer = t5_question_answering(content, question)
# print("Ref Files :" , related_files[0]["related_files"])
# print("Answer:", answer)

In [8]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/')
def hello():
    return 'Ready to connect to with API!'

@app.route('/Chat' , methods=['GET'])
def chat():
    response={}
    question = request.args.get('question', '')
    # question ="protokoll kommunfullmäktige 2024-02-29 signatur näe ?"
    related_files  = find_related_files(question, keywords_per_file)
    print("Related files:", related_files)
    content = related_files[0]["text"] if related_files else "Not relevant"
    rel_file= related_files[0]["related_files"] if related_files else "No file"
    answer = t5_question_answering(content, question)
    response["answer"]=answer
    response["related_files"]=rel_file
    return jsonify(response), 200

if __name__ == '__main__':
    # Start ngrok and create a tunnel
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)

    # Run Flask app
    app.run()


Public URL: https://492d-34-31-4-186.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Related files: [{'related_files': 'https://filer.offentligabeslut.se/media/ska/ska_hog/f/2024-02-29/1561ef3a-fd52-43d8-a6d4-a64a61a40d82.pdf', 'text': 'protokoll kommunfullmäktige 2024-02-29 signatur närvarande beslutande ledamöter péter kovács (m) (ordförande) ulf molin (c) (vice ordförande) wivi-anne broberg (s) (2:e vice ordförande) peter schölander (m) pia möller (m) marie tidedal (m) sandra kovács (m) lars linderot (m) bo caperman (m) boel olsson litström (m) gustaf wingårdh (m) nils hyllienmark (m) anita söderlind (m) fredrik walderyd (m) margareta widell (m) olof suneson (m) charlotte lewenhaupt (m) gertrud greén (c) pia lidwall (kd) johan ingvarson (mp) maria jönsson (mp) göran lock (mp) lennart nilsson (s) reinhold knutsson (s) carola persson (s) peter graff (s) heléne nyholm (s) alexander malmqvist (s) ann-magreth larsson (s) elisabeth eriksson (l) ros-mari paulsson (l) mikael bendz (l) christian johansson (sd) sixten paulsson (sd) lars metsäketo (sd) lars-erik nilsson (sd) j

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 17:58:20] "GET /Chat?question=protokoll%20kommunfullmäktige%202024-02-29%20signatur%20näe%20? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 17:59:00] "OPTIONS /Chat?question=protokoll%20kommunfullmäktige%202024-02-29%20signatur%20näe%20? HTTP/1.1" 200 -


Related files: [{'related_files': 'https://filer.offentligabeslut.se/media/ska/ska_hog/f/2024-02-29/1561ef3a-fd52-43d8-a6d4-a64a61a40d82.pdf', 'text': 'protokoll kommunfullmäktige 2024-02-29 signatur närvarande beslutande ledamöter péter kovács (m) (ordförande) ulf molin (c) (vice ordförande) wivi-anne broberg (s) (2:e vice ordförande) peter schölander (m) pia möller (m) marie tidedal (m) sandra kovács (m) lars linderot (m) bo caperman (m) boel olsson litström (m) gustaf wingårdh (m) nils hyllienmark (m) anita söderlind (m) fredrik walderyd (m) margareta widell (m) olof suneson (m) charlotte lewenhaupt (m) gertrud greén (c) pia lidwall (kd) johan ingvarson (mp) maria jönsson (mp) göran lock (mp) lennart nilsson (s) reinhold knutsson (s) carola persson (s) peter graff (s) heléne nyholm (s) alexander malmqvist (s) ann-magreth larsson (s) elisabeth eriksson (l) ros-mari paulsson (l) mikael bendz (l) christian johansson (sd) sixten paulsson (sd) lars metsäketo (sd) lars-erik nilsson (sd) j

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 17:59:03] "GET /Chat?question=protokoll%20kommunfullmäktige%202024-02-29%20signatur%20näe%20? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 17:59:16] "OPTIONS /Chat?question=WHo%20am%20i HTTP/1.1" 200 -


Related files: []


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 17:59:18] "GET /Chat?question=WHo%20am%20i HTTP/1.1" 200 -
